<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 4. MLP avanzado</font></h1>

<h1><font color="#113D68" size=4>4. Dropout para reducir el overfitting</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Introducción](#section1)
* [2. Regularización por dropout](#section2)
* [3. Dropout en la capa visible](#section3)
* [4. Dropout en capas ocultas](#section4)
* [5. Consejos al utilizar Dropout](#section5)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En esta lección, descubrirá la técnica de regularización de _Dropout_ y cómo aplicarla. Después de completar esta lección, sabrá:
* Cómo funciona la técnica de regularización de _Dropout._
* Cómo utilizar _Dropout_ en sus capas de entrada.
* Cómo utilizar _Dropout_ en sus capas ocultas.

In [2]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Introducción</font>

Dropout es una técnica en la que se ignoran las neuronas seleccionadas al azar durante el entrenamiento. Se eliminan al azar. Esto significa que su contribución a la activación de las neuronas siguientes se elimina temporalmente en el pase hacia adelante y las actualizaciones de peso no se aplican a la neurona en el pase hacia atrás.

Esta dependencia del contexto de una neurona durante el entrenamiento se denomina _coadaptaciones complejas._ Entonces si las neuronas se eliminan aleatoriamente, otras neuronas tendrán que intervenir y manejar la representación requerida para hacer predicciones para las neuronas faltantes.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el artículo [Dropout: A Simple Way to Prevent Neural Networks from Overfitting](https://jmlr.org/papers/v15/srivastava14a.html)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Regularización por Dropout</font>

_Dropout_ se implementa fácilmente seleccionando aleatoriamente los nodos que se eliminarán con una probabilidad determinada (por ejemplo, 20%) en cada ciclo de actualización de peso.

Los ejemplos utilizarán el conjunto de datos de clasificación binaria del conjunto de datos de Sonar. 

El modelo de red neuronal de línea de base tiene:
1. Dos capas ocultas, la primera con 60 neuronas y la segunda con 30. 
2. Gradiente Descendiente Estocástico se utiliza para el entrenamiento con una tasa de aprendizaje y _momentum_ relativamente bajos. 

Veamos los resultados de linea base sin _Dropout._

In [10]:
# Baseline Model on the Sonar Dataset
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# load dataset
r = "C:/Users/Maria Luisa/OneDrive/Documentos/Cursos/DeepLearningConPythonyKerasRedesNeuronalesAvanzado/DropOut_para_generalizarmodelo/Datasets/sonar.csv"
df = pd.read_csv(r, header=None)
df_values = df.values
# split into input (X) and output (Y) variables
X = df_values[:, 0:60].astype(float)
y = df_values[:, 60]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y =encoder.transform(y)

# baseline
def baseline_model():
    model = Sequential()
    model.add(Dense(60, input_dim = 60, activation = 'relu'))
    model.add(Dense(30, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    
    sgd = SGD(lr = 0.01, momentum = 0.8)
    model.compile(loss='binary_crossentropy', optimizer = sgd, metrics=['accuracy'])
    return model
    
# Results
estimator = []
estimator.append(('standarize', StandardScaler()))
estimator.append(('MLP', KerasClassifier(build_fn = baseline_model,
                                        epochs=300, 
                                        batch_size=16,
                                        verbose=0)))
pipeline= Pipeline(estimator)
kfold = StratifiedKFold(n_splits= 10, shuffle=True)

results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print('Modelo de linea base: %.2f%% (%.2f%%)'%(results.mean()*100, 
                                              results.std()*100))


Modelo de linea base: 85.55% (8.31%)


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Dropout en la capa visible</font>

_Dropout_ se puede aplicar a las neuronas de entrada. En el siguiente ejemplo: 

1. Agregamos una nueva capa de exclusión entre la entrada (o capa visible) y la primera capa oculta. 
2. La tasa de _Dropout_ se establece en 20%, i.e., una de cada cinco entradas se excluirá al azar.
3. Se impone una restricción en los pesos para cada capa oculta con la norma máxima de los pesos para que no exceda de 3. 
    * Esto se hace estableciendo el argumento de `kernel_constraint` en el clase `Dense` al construir las capas. 
4. Aumentamos la tasa de aprendizaje y momentum. 

Vemoas el código

In [8]:
# Example of Dropout on the Sonar Dataset: Visible Layer
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# load dataset
r = "C:/Users/Maria Luisa/OneDrive/Documentos/Cursos/DeepLearningConPythonyKerasRedesNeuronalesAvanzado/DropOut_para_generalizarmodelo/Datasets/sonar.csv"
df = pd.read_csv(r, header=None)
df_values = df.values
# split into input (X) and output (Y) variables
X = df_values[:, 0:60].astype(float)
y = df_values[:, 60]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y =encoder.transform(y)

# baseline
def input_dropout_model():
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(60,)))
    model.add(Dense(60, activation = 'relu',
                    kernel_constraint = maxnorm(3)))
    model.add(Dense(30, activation = 'relu',
                   kernel_constraint = maxnorm(3)))
    model.add(Dense(1, activation = 'sigmoid'))
    
    sgd = SGD(lr = 0.01, momentum = 0.9)
    model.compile(loss='binary_crossentropy', optimizer = sgd,
                  metrics=['accuracy'])
    return model
    
# Results
estimator = []
estimator.append(('standarize', StandardScaler()))
estimator.append(('MLP', KerasClassifier(build_fn = input_dropout_model,
                                        epochs=300, 
                                        batch_size=16,
                                        verbose=0)))
pipeline= Pipeline(estimator)
kfold = StratifiedKFold(n_splits= 10, shuffle=True)


results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print('Modelo con Dropout en la entrada: %.2f%% (%.2f%%)'%(results.mean()*100, 
                                              results.std()*100))


Modelo con Dropout en la entrada: 88.86% (10.38%)


<a id="section4"></a>
# <font color="#004D7F" size=6>4. Dropout en capas ocultas</font>

_Dropout_ se aplica también a las capas ocultas. En el siguiente ejemplo:

1. _Dropout_ se aplica entre las dos capas ocultas y entre la última capa oculta y la capa de salida. 
2. Nuevamente, se usa una tasa de _Dropout_ del 20%.

In [9]:
# Example of Dropout on the Sonar Dataset: Visible Layer
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# load dataset
r = "C:/Users/Maria Luisa/OneDrive/Documentos/Cursos/DeepLearningConPythonyKerasRedesNeuronalesAvanzado/DropOut_para_generalizarmodelo/Datasets/sonar.csv"
df = pd.read_csv(r, header=None)
df_values = df.values
# split into input (X) and output (Y) variables
X = df_values[:, 0:60].astype(float)
y = df_values[:, 60]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y =encoder.transform(y)

# baseline
def hidden_dropout_model():
    model = Sequential()
    model.add(Dense(60, input_dim=60, activation = 'relu',
                    kernel_constraint = maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(30, activation = 'relu',
                   kernel_constraint = maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = 'sigmoid'))
    
    sgd = SGD(lr = 0.01, momentum = 0.9)
    model.compile(loss='binary_crossentropy', optimizer = sgd,
                  metrics=['accuracy'])
    return model
    
# Results
estimator = []
estimator.append(('standarize', StandardScaler()))
estimator.append(('MLP', KerasClassifier(build_fn = hidden_dropout_model,
                                        epochs=300, 
                                        batch_size=16,
                                        verbose=0)))
pipeline= Pipeline(estimator)
kfold = StratifiedKFold(n_splits= 10, shuffle=True)


results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print('Modelo con Dropout en las capas ocultas: %.2f%% (%.2f%%)'%(results.mean()*100, 
                                              results.std()*100))



Modelo con Dropout en la entrada: 85.12% (8.34%)


<a id="section5"></a>
# <font color="#004D7F" size=6>5. Consejos al utilizar Dropout</font>

El documento original sobre _Dropout_ proporciona resultados experimentales sobre un conjunto de problemas estándar de Machine Learning. Como resultado, proporcionan una serie de heurísticas útiles a considerar cuando se usa la deserción en la práctica:
* Por lo general, utilice un pequeño valor de _Dropout_ del 20% al 20%-50% como punto de partida.
* Utilizar una red más grande.
* Utilizar _Dropout_ en la entrada (capa visible) y capas ocultas. 
* Utilizar una gran tasa de aprendizaje (aumentar el factor de 10 a 100) y un gran momentum (0.9 a 0.99). 
* Restrinja el tamaño de los pesos de la red. Una gran tasa de aprendizaje puede resultar en pesos de red muy grandes. 

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>